# Yotuubef Colab Notebook
This notebook clones the yotuubef repository, installs dependencies, and runs the script with GPU acceleration (T4).

## One-Click Setup
1. **Run all cells** (Runtime > Run all)
2. When prompted, upload a single `.env` file containing:
   ```
   # YouTube API
   YOUTUBE_CLIENT_ID="your_youtube_client_id"
   YOUTUBE_CLIENT_SECRET="your_youtube_client_secret"
   YOUTUBE_PROJECT_ID="your_youtube_project_id"
   
   # Reddit API
   REDDIT_CLIENT_ID="your_reddit_id"
   REDDIT_CLIENT_SECRET="your_reddit_secret"
   
   # Gemini API
   GEMINI_API_KEY="your_gemini_key"
   ```
3. The script will automatically:
   - Generate credentials.json from the .env file
   - Set all environment variables
   - Authenticate with YouTube
   - Process videos
   - Upload to YouTube

> **Note**: Get API keys from:
> - YouTube: https://console.cloud.google.com/apis/credentials
> - Reddit: https://www.reddit.com/prefs/apps
> - Gemini: https://aistudio.google.com/app/apikey

In [ ]:
# Set up T4 GPU
!nvidia-smi --query-gpu=name --format=csv,noheader

In [ ]:
# Clone repository
!git clone https://github.com/beenycool/yotuubef
%cd yotuubef

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Upload unified .env file
from google.colab import files
import os
import json
import shutil
from datetime import datetime

# Create secure_files directory
if not os.path.exists('secure_files'):
    os.makedirs('secure_files')

# Upload .env file
uploaded = files.upload()
env_uploaded = False

for filename in uploaded.keys():
    # Only process .env files
    if filename.endswith('.env'):
        # Move to secure_files
        shutil.move(filename, f'secure_files/{filename}')
        print(f"Uploaded: {filename}")
        env_uploaded = True
        
        # Load environment variables
        with open(f'secure_files/{filename}') as f:
            for line in f:
                if line.strip() and not line.startswith('#'):
                    key, value = line.strip().split('=', 1)
                    os.environ[key] = value.strip('"')
        
        # Create token file if YouTube credentials exist
        if all(k in os.environ for k in ['YOUTUBE_TOKEN', 'YOUTUBE_REFRESH_TOKEN']):
            token_data = {
                "token": os.environ['YOUTUBE_TOKEN'],
                "refresh_token": os.environ['YOUTUBE_REFRESH_TOKEN'],
                "token_uri": os.environ.get('YOUTUBE_TOKEN_URI', 'https://oauth2.googleapis.com/token'),
                "client_id": os.environ['YOUTUBE_CLIENT_ID'],
                "client_secret": os.environ['YOUTUBE_CLIENT_SECRET'],
                "scopes": os.environ.get('YOUTUBE_SCOPES', '').split(','),
                "expiry": os.environ.get('YOUTUBE_EXPIRY', datetime.utcnow().isoformat())
            }
            
            with open('secure_files/youtube_token.json', 'w') as f:
                json.dump(token_data, f, indent=2)
            os.environ['YOUTUBE_TOKEN_FILE'] = 'secure_files/youtube_token.json'
            print("YouTube token configured")
        
        # Automate OAuth flow
        print("Starting YouTube authentication...")
        !python -c "from script import setup_api_clients; setup_api_clients()"
        print("YouTube authentication complete!")
        
        # Copy .env to root for dotenv
        shutil.copy(f'secure_files/{filename}', '.env')
        print("Environment variables configured")
        
        break

if not env_uploaded:
    print("Warning: No .env file uploaded. The script may not work properly")

In [ ]:
# Run main script
!python script.py